In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into 

In [4]:
chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into 